# Actividad 5.1 Matriz HAT

Iván Serrano González - A01748517

Juan Emilio de la Torre Bárcenas - A01708606

# EJERCICIO 2

## Verificar los resultados utilizando Python

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Definir los datos
X = np.array([[1, 2],
              [1, 3],
              [1, 5],
              [1, 7],
              [1, 9]])
y = np.array([5, 8, 7, 10, 12])

# Ajustar el modelo
reg = LinearRegression(fit_intercept=False)
reg.fit(X, y)

# Predicciones del modelo sklearn
y_pred_sklearn = reg.predict(X)

# Coeficientes del modelo sklearn
coef_sklearn = reg.coef_

print("\nVerificación:")
print("Coeficientes (sklearn):", coef_sklearn)
print("Valores predichos (sklearn):", y_pred_sklearn)

# EJERCICIO 3

#### 3. Utilizando los Datos “Cirugía de Hígado” obtener la matriz HAT, los valores predichos, y, y los coeficientes de la ecuación de regresión β utilizando el método de matrices. (Puede realizarse con cualquier paquete).

In [17]:
#Importar Librerías
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [16]:
#Cargar los datos desde un archivo CSV
file_path = 'data.csv'
data = pd.read_csv(file_path, encoding='latin1')

In [13]:
data

,Factor Coagulacion,Indice pronostico,Funcion de enzima,Funcion de higado,Edad,Genero,Alcohol (moderado),Alcohol (severo),Sobrevivencia (dias)
0,6.7,62,81,2.59,50,0,1,0,695
1,5.1,59,66,1.70,39,0,0,0,403
2,7.4,57,83,2.16,55,0,0,0,710
3,6.5,73,41,2.01,48,0,0,0,349
4,7.8,65,115,4.30,45,0,0,1,2343
...,...,...,...,...,...,...,...,...,...
103,5.8,70,64,2.52,49,0,1,0,589
104,5.4,64,81,1.36,62,0,1,0,599
105,6.9,90,33,2.78,48,1,0,0,655
106,7.9,45,55,2.46,43,0,1,0,377


In [20]:
#Seleccionar las columnas de características para el modelo
X = data[['Factor Coagulacion', 'Indice pronostico', 'Funcion de enzima',
          'Funcion de higado', 'Edad', 'Genero',
          'Alcohol (moderado)', 'Alcohol (severo)']]

#Seleccionar la variable dependiente
y = data['Sobrevivencia (dias)']

In [21]:
#Normalizar las características usando MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

#Añadir una columna de unos (intercepto) a la matriz X normalizada
X_scaled = np.hstack([np.ones((X_scaled.shape[0], 1)), X_scaled])

#Calcular la inversa de la matriz XtX (X_transpuesta * X) usando álgebra matricial
XtX_inv = np.linalg.inv(X_scaled.T @ X_scaled)
beta = XtX_inv @ X_scaled.T @ y

#Calcular los valores predichos de la variable dependiente (y_hat)
y_hat = X_scaled @ beta

#Calcular la matriz HAT (proyección), que ayuda a identificar puntos influyentes
H = X_scaled @ XtX_inv @ X_scaled.T

#Crear un DataFrame con los coeficientes obtenidos de la regresión
coef_df = pd.DataFrame(beta, index=['Intercepto'] + list(data.columns[:-1]), columns=['Coeficientes'])

#Crear un DataFrame con los valores predichos (y_hat)
y_hat_df = pd.DataFrame(y_hat, columns=['Valores Predichos'])

#Crear un DataFrame para la matriz HAT con nombres de columnas personalizados
H_df = pd.DataFrame(H, columns=[f'Col_{i+1}' for i in range(H.shape[1])])
H_df_limited = H_df.iloc[:20, :20]

#Imprimir resultados
print("Coeficientes de regresión:")
print(coef_df.round(4))

print("\nValores predichos:")
print(y_hat_df.round(4))

print("\nMatriz HAT (primeros 20x20):")
print(H_df_limited.round(4))

Coeficientes de regresión:
                    Coeficientes
Intercepto             -575.8637
Factor Coagulacion      453.2609
Indice pronostico       738.2324
Funcion de enzima       854.1430
Funcion de higado       429.4316
Edad                     25.6389
Genero                   13.0926
Alcohol (moderado)      -41.2676
Alcohol (severo)        195.7070

Valores predichos:
     Valores Predichos
0             706.2562
1             430.8229
2             732.2298
3             425.0396
4            1454.5876
..                 ...
103           566.5151
104           568.3349
105           584.3679
106           385.9099
107           459.7200

[108 rows x 1 columns]

Matriz HAT (primeros 20x20):
     Col_1   Col_2   Col_3   Col_4   Col_5   Col_6   Col_7   Col_8   Col_9  \
0   0.0371  0.0128  0.0315  0.0105  0.0233  0.0132 -0.0160 -0.0015  0.0397   
1   0.0128  0.0611  0.0483  0.0488 -0.0010 -0.0099 -0.0085 -0.0247  0.0092   
2   0.0315  0.0483  0.0951  0.0493  0.0283  0.0145 -0.0181 

#### 4. Concluir sobre el significado de los valores de apalancamiento y formular la ecuación de regresión.

Los valores de apalancamiento indican cuánta influencia tiene un punto de datos en el modelo de regresión. Si un punto tiene un valor alto, significa que tiene un impacto significativo en la línea de regresión, y podría ser un outlier. En general, se busca que la mayoría de los puntos tengan valores de apalancamiento moderados para evitar que unos pocos datos distorsionen los resultados del modelo.

La ecuación de regresión final es:
ŷ = -575.8637 + 453.2609 * (Factor Coagulación) + 738.2324 * (Índice pronóstico) + 854.1430 * (Función de enzima) + 429.4316 * (Función de hígado) + 25.6389  * (Edad) + 13.0926  * (Género) - 41.2676  * (Alcohol moderado) + 195.7070 * (Alcohol severo)

Los coeficientes reflejan el impacto que cada variable tiene sobre la sobrevivencia, donde variables como la **función de enzima** y el **índice pronóstico** tienen una influencia positiva significativa, mientras que el consumo **moderado de alcohol** tiene un efecto negativo.